In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [2]:
#We want to use a resnet50 from torchvision to have the embedding of an image, use a pretrained resnet and remove the last layer

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.resnet = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        self.resnet.eval()
        for param in self.resnet.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.resnet(x)

In [3]:
#Define a clustering model that will take the embeddings and output the clusters and the cluster centers

class ClusteringModel(nn.Module):
    def __init__(self, n_clusters):
        super(ClusteringModel, self).__init__()
        self.n_clusters = n_clusters
        self.cluster_centers = nn.Parameter(torch.randn(n_clusters, 512))
        self.fc = nn.Linear(512, n_clusters)

    def forward(self, x):
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        cluster_centers = self.cluster_centers
        return x, cluster_centers

In [4]:
#Define a KNN model that will take the embeddings and output the clusters and the cluster centers

class KNNModel(nn.Module):
    def __init__(self, n_clusters):
        super(KNNModel, self).__init__()
        self.n_clusters = n_clusters
        self.cluster_centers = nn.Parameter(torch.randn(n_clusters, 512))

    def forward(self, x):
        x = x.unsqueeze(1)
        cluster_centers = self.cluster_centers.unsqueeze(0)
        distances = torch.norm(x - cluster_centers, dim=2, p=None)
        clusters = torch.argmin(distances, dim=1)
        return clusters, self.cluster_centers

In [5]:
# Do a simple KNN for an image embedding

class KNN(nn.Module):
    def __init__(self, k):
        super(KNN, self).__init__()
        self.k = k
    def forward(self, dataset, x):
        distances = torch.cdist(x, dataset)

        # Get indices of k nearest neighbors for each test point
        _, indices = torch.topk(distances, self.k, largest=False)
        return indices


In [6]:
#Create the resnet model
resnet = ResNet()


Using cache found in /Users/jskaf/.cache/torch/hub/pytorch_vision_v0.6.0


In [7]:
from dataset import myDataset

#Create the dataset

get_preprocessed_image = True
my_path_hm = os.path.join(os.getcwd(), 'data/h&mdataset/images/')
my_path_fash = os.path.join(os.getcwd(), 'data/fashion-dataset/images/')

dataset = myDataset(my_path_hm, my_path_fash, get_preprocessed_image, 'both')


AttributeError: module 'torchvision.models' has no attribute 'ResNet50_Weights'